# 🔄 Preprocessing Data: Cleaning & Preparation 🛠️  
Setelah berhasil melakukan **scraping** data ulasan dari Google Play Store, langkah selanjutnya adalah **preprocessing**.  
Tahap ini bertujuan untuk membersihkan dan menyiapkan data agar siap untuk analisis lebih lanjut.  

---

## 🔧 **Libraries Used**

In [118]:
import pandas as pd
import contractions
from transformers import BertTokenizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string
from symspellpy import SymSpell, Verbosity
from collections import Counter
import langid

## 📥 **Import Data CSV**

In [119]:
# Import data
file_path = "../data/1_scrapping_running_tracker.csv"
df = pd.read_csv(file_path)
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,aaa7089c-2edc-46de-84d1-ef6434a011f2,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,I really like it,5,0,1.7.4,2025-03-12 09:42:30,NaN,NaN,1.7.4
1,a3f09ed5-3e22-417a-9f95-f80c1fa752ea,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Excellent,5,0,1.7.5,2025-03-12 07:20:52,NaN,NaN,1.7.5
2,a02fb34e-14a1-4345-8b63-dc3b9ddd1f74,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Good,4,0,1.7.4,2025-03-12 06:48:45,Your praise is the greatest encouragement to u...,2025-03-12 14:16:05,1.7.4
3,958e76aa-ba71-4345-96c3-7578fa43126b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,👍,5,0,1.7.4,2025-03-12 06:35:37,NaN,NaN,1.7.4
4,0be62035-3d04-4d3d-b2a6-9efd5c48cbab,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,You can't swipe away notifications,1,0,1.7.4,2025-03-12 06:17:30,"Hi, thanks for your feedback. Could you send u...",2025-03-12 14:47:29,1.7.4


## 📌 **Tahapan Preprocessing**

### ✅ **1. Handling Missing Values** → Menghapus atau mengisi data yang kosong  

In [120]:
print(df.isnull().sum())

reviewId                    0
userName                    0
userImage                   0
content                     1
score                       0
thumbsUpCount               0
reviewCreatedVersion     2328
at                          0
replyContent            27722
repliedAt               27722
appVersion               2328
dtype: int64


Setelah melakukan pengecekan missing values, ditemukan beberapa kolom dengan nilai kosong:  

| **Kolom**               | **Jumlah Missing Values** | **Penanganan** |
|-------------------------|-------------------------|---------------|
| `content`              | 1                         | **Dihapus** (karena hanya 1 data) |
| `reviewCreatedVersion` | 2,328                     | **Diisi** dengan `"Unknown"` atau modus |
| `replyContent` & `repliedAt` | 27,722          | **Diisi** dengan `"No Reply"` |
| `appVersion`           | 2,328                     | **Diisi** dengan modus atau `"Unknown"` |

---

In [121]:
# Menghapus baris dengan content kosong
df = df.dropna(subset=["content"])  

# Menangani missing values dengan assignment langsung
df["reviewCreatedVersion"] = df["reviewCreatedVersion"].fillna("Unknown")
df["replyContent"] = df["replyContent"].fillna("No Reply")
df["repliedAt"] = df["repliedAt"].fillna("No Reply")
df["appVersion"] = df["appVersion"].fillna("Unknown")

In [122]:
print(df.isnull().sum())

reviewId                0
userName                0
userImage               0
content                 0
score                   0
thumbsUpCount           0
reviewCreatedVersion    0
at                      0
replyContent            0
repliedAt               0
appVersion              0
dtype: int64


### ✅ **2. Expand Contractions** → Mengubah singkatan

Expand contractions dilakukan sebelum tokenisasi karena banyak singkatan dalam bahasa Inggris yang mengandung apostrof (`'`), seperti `"can't"` menjadi `"cannot"` atau `"you're"` menjadi `"you are"`. Jika tokenisasi dilakukan terlebih dahulu, kata yang mengandung apostrof dapat terpisah menjadi token yang tidak bermakna, seperti `"can't"` menjadi `["ca", "n't"]`. Dengan melakukan ekspansi terlebih dahulu, kita memastikan bahwa setiap kata tetap utuh sebelum diproses lebih lanjut dalam analisis teks, sehingga hasil tokenisasi menjadi lebih akurat dan sesuai dengan makna sebenarnya.

In [123]:
def expand_contractions(text):
    """Mengubah singkatan dalam teks menjadi bentuk lengkapnya."""
    return contractions.fix(text)

df["content_expanded"] = df["content"].apply(expand_contractions)
df[["content", "content_expanded"]].head()

,content,content_expanded
0,I really like it,I really like it
1,Excellent,Excellent
2,Good,Good
3,👍,👍
4,You can't swipe away notifications,You cannot swipe away notifications


### ✅ **3. Tokenization** → Memisahkan teks menjadi kata-kata atau unit kecil lainnya.

Dalam proyek ini, tokenisasi dilakukan **menggunakan BERT tokenizer** untuk memproses teks ulasan aplikasi sebelum analisis sentimen. BERT (Bidirectional Encoder Representations from Transformers) dipilih karena memiliki **bidirectional context** , yang memungkinkan model memahami makna kata berdasarkan keseluruhan kalimat, serta **pre-trained model**, yang telah dilatih dengan miliaran kata sehingga meningkatkan akurasi pemrosesan teks. Selain itu, BERT sangat **cocok untuk analisis sentimen**, karena mampu menangkap nuansa emosi dalam teks lebih baik dibandingkan metode tokenisasi tradisional. 

In [124]:
# Tokenisasi menggunakan BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
df["tokens"] = df["content_expanded"].apply(lambda x: tokenizer.tokenize(x))

In [125]:
# Cek hasil tokenisasi
df[["content_expanded", "tokens"]].head()

,content_expanded,tokens
0,I really like it,"[i, really, like, it]"
1,Excellent,[excellent]
2,Good,[good]
3,👍,[[UNK]]
4,You cannot swipe away notifications,"[you, cannot, sw, ##ipe, away, notification, ##s]"


In [126]:
# Menggabungkan kata yang terpisah oleh ##

def merge_subwords(tokens):
    merged_tokens = []
    current_word = ""

    for token in tokens:
        if token.startswith("##"):
            current_word += token[2:]  # Gabungkan tanpa '##'
        else:
            if current_word:
                merged_tokens.append(current_word)  # Simpan kata sebelumnya
            current_word = token  # Mulai kata baru

    if current_word:  # Pastikan kata terakhir masuk
        merged_tokens.append(current_word)

    return merged_tokens

df["tokens_merged"] = df["tokens"].apply(merge_subwords)
df[["tokens", "tokens_merged"]].head()

,tokens,tokens_merged
0,"[i, really, like, it]","[i, really, like, it]"
1,[excellent],[excellent]
2,[good],[good]
3,[[UNK]],[[UNK]]
4,"[you, cannot, sw, ##ipe, away, notification, ##s]","[you, cannot, swipe, away, notifications]"


### ✅ **4. Lowercasing** → Mengubah teks menjadi huruf kecil untuk konsistensi

In [127]:
df["tokens_merged"] = df["tokens_merged"].apply(lambda x: [token.lower() for token in x])

# Cek hasil lowercasing
df[["tokens_merged", "tokens_merged"]].head()

,tokens_merged,tokens_merged
0,"[i, really, like, it]","[i, really, like, it]"
1,[excellent],[excellent]
2,[good],[good]
3,[[unk]],[[unk]]
4,"[you, cannot, swipe, away, notifications]","[you, cannot, swipe, away, notifications]"


### ✅ **5. Stopword Removal** → Menghapus kata-kata umum yang tidak bermakna dalam analisis

In [128]:
# nltk.download("stopwords")

stop_words = set(stopwords.words("english"))
important_words = {"not", "no", "never", "nor", "n't"}  
filtered_stop_words = stop_words - important_words  

def remove_stopwords(tokens):
    cleaned_tokens = [
        token.strip() for token in tokens  # Hapus spasi/karakter aneh
        if token.lower() not in filtered_stop_words and len(token) > 2  # Hapus hanya stop words yang tidak penting
    ]
    return cleaned_tokens

df["tokens_cleaned"] = df["tokens_merged"].apply(remove_stopwords)
df[["tokens_merged", "tokens_cleaned"]].head()


,tokens_merged,tokens_cleaned
0,"[i, really, like, it]","[really, like]"
1,[excellent],[excellent]
2,[good],[good]
3,[[unk]],[[unk]]
4,"[you, cannot, swipe, away, notifications]","[cannot, swipe, away, notifications]"


### ✅ **6.  Lemmatization** → Mengubah kata menjadi bentuk dasarnya

Dalam proses ini, kita menggunakan **Lemmatization** untuk mengubah kata ke bentuk dasarnya berdasarkan konteks linguistik. Berbeda dengan **Stemming** yang hanya memotong kata tanpa memperhatikan makna, **Lemmatization mempertahankan makna asli kata**. **BERT Tokenizer sudah memahami konteks kata**, sehingga proses Stemming justru bisa menghilangkan informasi penting yang diperlukan dalam analisis sentimen.  Jadi, proses stemming dilewati.


In [130]:
nltk.download("wordnet")

lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token, pos="v") for token in tokens]  # 'v' untuk verb

df["tokens_lemmatized"] = df["tokens_cleaned"].apply(lemmatize_tokens)
df[["tokens_cleaned", "tokens_lemmatized"]].head()

,tokens_cleaned,tokens_lemmatized
0,"[really, like]","[really, like]"
1,[excellent],[excellent]
2,[good],[good]
3,[[unk]],[[unk]]
4,"[cannot, swipe, away, notifications]","[cannot, swipe, away, notifications]"


### ✅ **7. Punctuation & Special Characters Removal** → Menghapus tanda baca dan simbol yang tidak relevan.

In [131]:
def clean_tokens(tokens):
    """Menghapus tanda baca dan token [UNK]"""
    return [token for token in tokens if token not in string.punctuation and token.lower() != "[unk]"]

df["tokens_cleaned"] = df["tokens_lemmatized"].apply(clean_tokens)
df[["tokens_lemmatized", "tokens_cleaned"]].head()

,tokens_lemmatized,tokens_cleaned
0,"[really, like]","[really, like]"
1,[excellent],[excellent]
2,[good],[good]
3,[[unk]],[]
4,"[cannot, swipe, away, notifications]","[cannot, swipe, away, notifications]"


### ✅ **8. Spelling Correction** → Memperbaiki kesalahan ejaan.

Untuk meningkatkan akurasi analisis teks, saya menggunakan **SymSpell** sebagai metode *spelling correction*. SymSpell adalah algoritma koreksi ejaan yang sangat cepat karena menggunakan teknik pencocokan berbasis hash dan precomputed edit distance.  

**kamus frekuensi kata berbahasa Inggris** dari repositori SymSpell:  
[SymSpell Frequency Dictionary](https://github.com/wolfgarbe/SymSpell/blob/master/SymSpell/frequency_dictionary_en_82_765.txt)  


In [132]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Load dictionary
dictionary_path = "frequency_dictionary_en_82_765.txt"  
if not sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1):
    print("Gagal memuat dictionary. Pastikan file dictionary tersedia di path yang benar.")

def correct_spelling(tokens):
    corrected_tokens = []
    for token in tokens:
        suggestion = sym_spell.lookup(token, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestion:
            corrected_tokens.append(suggestion[0].term)
        else:
            corrected_tokens.append(token)  # Jika tidak ada koreksi, gunakan kata asli
    return corrected_tokens

# Terapkan koreksi ejaan pada token yang sudah dibersihkan
df["tokens_spelling_corrected"] = df["tokens_cleaned"].apply(correct_spelling)

# Tampilkan hanya baris yang mengalami perubahan
df_changes = df[df["tokens_cleaned"].astype(str) != df["tokens_spelling_corrected"].astype(str)]
df_changes[["tokens_cleaned", "tokens_spelling_corrected"]].head()


,tokens_cleaned,tokens_spelling_corrected
9,"[veryyy, goood]","[very, good]"
10,"[nice, feature, simple, use, dependable, gps]","[nice, feature, simple, use, dependable, gas]"
12,"[nice, aap]","[nice, map]"
37,"[nice, best, app, run, ads, funcions, overall,...","[nice, best, app, run, ads, functions, overall..."
48,"[good, app, keep, calculate, wrond, distance, ...","[good, app, keep, calculate, wrong, distance, ..."


### ✅ **9. Rare Words Removal** → Menghapus kata yang jarang muncul.

Pada tahap ini, kata-kata yang jarang muncul dalam dataset akan dihapus. Rare words sering kali tidak memberikan kontribusi signifikan dalam analisis sentimen karena frekuensinya yang sangat rendah dan cenderung menjadi noise.  

In [133]:
word_counts = Counter(word for tokens in df["tokens_spelling_corrected"] for word in tokens)

# Tentukan threshold (kata muncul kurang dari 5 kali akan dihapus)
rare_threshold = 5  
rare_words = {word for word, count in word_counts.items() if count < rare_threshold}

# Fungsi untuk menghapus rare words dari token
def remove_rare_words(tokens):
    return [word for word in tokens if word not in rare_words]

df["tokens_rare_removed"] = df["tokens_spelling_corrected"].apply(remove_rare_words)

df_changes = df[df["tokens_spelling_corrected"].astype(str) != df["tokens_rare_removed"].astype(str)]
df_changes[["tokens_spelling_corrected", "tokens_rare_removed"]].head()

,tokens_spelling_corrected,tokens_rare_removed
4,"[cannot, swipe, away, notifications]","[cannot, away, notifications]"
10,"[nice, feature, simple, use, dependable, gas]","[nice, feature, simple, use, gas]"
17,"[weight, loss, bat, application]","[weight, loss, application]"
37,"[nice, best, app, run, ads, functions, overall...","[nice, best, app, run, ads, overall, free]"
55,"[exemplary, good]",[good]


Dalam analisis sentimen ulasan aplikasi, penghapusan kata-kata umum (Common Words Removal) **tidak dilakukan** karena:  

1. **Stopwords sudah dihapus lebih awal** → Kata-kata umum yang tidak bermakna sudah dihilangkan melalui proses stopwords removal.  
2. **Kata-kata umum dapat memiliki makna dalam sentimen** → Kata seperti *"good"*, *"bad"*, atau *"great"* penting untuk menentukan polaritas sentimen.  
3. **Menghindari kehilangan konteks** → Jika terlalu banyak kata dihapus, makna asli dari ulasan bisa berubah atau berkurang.  

### ✅ **10. Language Detection & Filtering** → Mendeteksi bahasa dan menghapus selain bahasa Inggris.

Langkah selanjutnya adalah **mendeteksi bahasa** pada setiap ulasan.  
Karena analisis ini difokuskan pada ulasan berbahasa Inggris, maka ulasan yang terdeteksi menggunakan bahasa lain akan **dihapus**.

In [134]:
def detect_language_langid(tokens):
    text = " ".join(tokens)
    return langid.classify(text)[0]  # Ambil kode bahasa 

df = df.copy()
df.loc[:, "language"] = df["tokens_rare_removed"].apply(detect_language_langid)

print(df["language"].value_counts())

df_non_english = df[df["language"] != "en"]
print(df_non_english[["content", "language"]].head())


language
en    24030
de     1218
sv     1041
nl      440
fr      425
es      288
no      214
it      183
nb      180
da      167
mt       94
ms       64
pl       61
fi       54
eu       50
lv       50
lb       49
id       42
fa       37
af       35
et       27
ca       23
ar       21
pt       21
ro       17
sl       13
nn       11
br        9
oc        9
tl        8
bg        7
ru        7
wa        5
hu        5
hi        5
sw        5
vi        4
eo        3
la        2
cs        2
is        2
ht        1
sk        1
cy        1
tr        1
ga        1
gl        1
lt        1
Name: count, dtype: int64
                                             content language
1                                          Excellent       de
8                                               خوبه       fa
17                     Weight loss bat application 😚       es
25  One of the best app, simple and straight forward       sv
26     very good app motivate everyday for running 😀       no


In [135]:
# Hapus baris yang bukan bahasa Inggris
df = df[df["language"] == "en"].copy()

# Tampilkan jumlah ulasan setelah penghapusan
print(f"Jumlah ulasan setelah filter hanya bahasa Inggris: {len(df)}")

Jumlah ulasan setelah filter hanya bahasa Inggris: 24030


### ✅ **11 .Delete Empty Rows**

Jika sebuah review kehilangan katanya setelah proses preprocessing ini, maka baris tersebut tidak lagi memiliki informasi yang berguna. Oleh karena itu, baris-baris kosong akan dihapus agar dataset tetap bersih dan bermakna untuk analisis sentimen.

In [136]:
# Cek jumlah baris yang memiliki token kosong setelah preprocessing
df_empty = df[df["tokens_rare_removed"].apply(lambda x: len(x) == 0)]
print(f"Jumlah baris kosong: {len(df_empty)}")

Jumlah baris kosong: 1515


In [137]:
# Hapus baris yang kosong
df = df[df["tokens_rare_removed"].apply(lambda x: len(x) > 0)].copy()

# Tampilkan jumlah ulasan setelah penghapusan
print(f"Jumlah ulasan setelah menghapus baris kosong: {len(df)}")

Jumlah ulasan setelah menghapus baris kosong: 22515


## 📌 **Save Data**

In [144]:
# Simpan hanya konten yang telah dibersihkan
df["content_cleaned"] = df["tokens_rare_removed"].apply(lambda x: " ".join(x))

# Simpan DataFrame ke dalam folder data dengan path relatif 
output_path = "../data/2_preprocess_running_tracker.csv"
df.to_csv(output_path, index=False, encoding="utf-8")

print(f"Data berhasil disimpan ke {output_path}")

Data berhasil disimpan ke ../data/2_preprocess_running_tracker.csv


In [145]:
columns_to_keep = ["reviewId", "userName", "userImage", "score", "thumbsUpCount", 
                   "reviewCreatedVersion", "at", "replyContent", "repliedAt", 
                   "appVersion", "content_cleaned"]  

# Buat DataFrame baru dengan hanya kolom yang diperlukan
df_final = df[columns_to_keep].copy()

# Simpan DataFrame ke dalam folder data dengan path relatif
output_path = "../data/2_cleandata_running_tracker.csv"
df_final.to_csv(output_path, index=False, encoding="utf-8")

print(f"Data berhasil disimpan ke {output_path}")

Data berhasil disimpan ke ../data/2_cleandata_running_tracker.csv
